In [ ]:
! pip install transformers[torch] datasets scikit-learn elasticsearch

In [1]:
import pandas as pd
import random

In [2]:
from datasets import load_dataset

ds_qa = load_dataset("clarin-knext/fiqa-pl-qrels")
ds_corpus = load_dataset("clarin-knext/fiqa-pl", "corpus")
ds_queries = load_dataset("clarin-knext/fiqa-pl", "queries")

/home/mklimek/venvs/pytorch-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd

ds_corpus['corpus'][:3]

{'_id': ['3', '31', '56'],
 'title': ['', '', ''],
 'text': ['Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. Szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. Być może systemy edukacyjne w Stanach Zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłużonych studentów i narzekać, że nie są do niczego wykwalifikowani.',
  'Tak więc nic nie zapobiega fałszywym ocenom poza dodatkową kontrolą ze strony rynku/inwestorów, ale istnieją pewne nowsze kontrole, które uniemożliwiają instytucjom korzystanie z nich. W ramach DFA banki nie mogą już polegać wyłącznie na ratingach kredytowych jako należytej staranności przy zakupie instrumentu finansowego, więc to jest plus. Intencją jest to, że jeśli instytucje finansowe wykonują swoją własną pracę, to *być może* dojdą do wniosku, że określony CDO

In [4]:
corpus_df = pd.DataFrame(ds_corpus['corpus'])
corpus_df.head(5)

,_id,title,text
0,3,,"Nie mówię, że nie podoba mi się też pomysł szk..."
1,31,,Tak więc nic nie zapobiega fałszywym ocenom po...
2,56,,Nigdy nie możesz korzystać z FSA dla indywidua...
3,59,,Samsung stworzył LCD i inne technologie płaski...
4,63,,Oto wymagania SEC: Federalne przepisy dotycząc...


In [5]:
ds_queries['queries'][:3]

{'_id': ['0', '4', '5'],
 'title': ['', '', ''],
 'text': ['Co jest uważane za wydatek służbowy w podróży służbowej?',
  'Wydatki służbowe - ubezpieczenie samochodu podlegające odliczeniu za wypadek, który wydarzył się podczas podróży służbowej',
  'Rozpoczęcie nowego biznesu online']}

In [6]:
queries_df = pd.DataFrame(ds_queries['queries'])
queries_df.head(5)

,_id,title,text
0,0,,Co jest uważane za wydatek służbowy w podróży ...
1,4,,Wydatki służbowe - ubezpieczenie samochodu pod...
2,5,,Rozpoczęcie nowego biznesu online
3,6,,„Dzień roboczy” i „termin płatności” rachunków
4,7,,Nowy właściciel firmy – Jak działają podatki d...


In [7]:
ds_qa['train'][:10]

{'query-id': [0, 4, 5, 6, 6, 6, 7, 9, 9, 11],
 'corpus-id': [18850,
  196463,
  69306,
  560251,
  188530,
  564488,
  411063,
  509122,
  184698,
  596427],
 'score': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [8]:
query_ids_set = set(ds_qa['train']['query-id'])
len(query_ids_set)

5500

In [9]:
import pandas as pd

qa_df = pd.DataFrame(ds_qa['train'])

In [10]:
q_id = 6
qa_related = list(qa_df.loc[qa_df['query-id'] == q_id]['corpus-id'])
print(qa_related)
random.choice(qa_related)

[560251, 188530, 564488]


188530

In [11]:
def prepare_related_sentences(query_ids, sep_token='|'):
    results = []
    for q_id in query_ids:
        qa_related = list(qa_df[qa_df['query-id'] == q_id]['corpus-id'])
        rel_id = random.choice(qa_related)
        q_text = queries_df[queries_df['_id'] == str(q_id)]['text'].iloc[0]
        rel_text = corpus_df[corpus_df['_id'] == str(rel_id)]['text'].iloc[0]
        results.append(q_text + sep_token + rel_text)

    return results

In [12]:
prepare_related_sentences([0, 4, 5])

['Co jest uważane za wydatek służbowy w podróży służbowej?|Wytyczne IRS dotyczące tematu. Ogólnie rzecz biorąc, najlepsze, co mogę powiedzieć, to to, że Twój wydatek biznesowy może podlegać odliczeniu. Ale to zależy od okoliczności i tego, co chcesz odliczyć. Podróże Podatnicy, którzy wyjeżdżają z domu w celach służbowych, mogą odliczyć związane z tym wydatki, w tym koszty dotarcia do miejsca docelowego, koszty zakwaterowania i wyżywienia oraz inne zwykłe i niezbędne wydatki. Podatnicy są uważani za „wyjeżdżających poza dom”, jeśli ich obowiązki wymagają od nich przebywania poza domem znacznie dłużej niż zwykły dzień pracy i muszą spać lub odpoczywać, aby sprostać wymogom pracy. Można odliczyć rzeczywisty koszt posiłków i nieprzewidziane wydatki lub skorzystać ze standardowej diety żywieniowej i obniżonych wymogów ewidencji. Niezależnie od zastosowanej metody odliczenia posiłków są zazwyczaj ograniczone do 50 procent, jak wspomniano wcześniej. Jako koszt można zgłaszać tylko rzeczywist

In [13]:
from transformers import AutoTokenizer

model_name = "dkleczek/bert-base-polish-cased-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
polbert_separator = tokenizer.sep_token
polbert_separator

'[SEP]'

In [41]:
N = 2500
related_sentences = prepare_related_sentences(random.sample(list(query_ids_set), N), polbert_separator)
related_sentences[:3]

['Kiedy powinienem zacząć oszczędzać/inwestować na emeryturę?[SEP]Zacznij jak najszybciej i zrób swoją rutynę oszczędzania. Zacznij od tego, z czym czujesz się komfortowo i bądź konsekwentny. Zwiększ tę kwotę za pomocą podwyżek, zysków i kiedy tylko chcesz.',
 'W jaki sposób obywatel spoza USA uzyskuje status akredytowanego inwestora SEC?[SEP]Oto wymagania SEC: Federalne przepisy dotyczące papierów wartościowych definiują termin inwestora akredytowanego w zasadzie 501 Regulacji D jako: bank, firma ubezpieczeniowa, zarejestrowana firma inwestycyjna, firma zajmująca się rozwojem działalności lub firma inwestycyjna dla małych firm; program świadczeń pracowniczych w rozumieniu ustawy o zabezpieczeniach emerytalnych pracowników, jeżeli decyzje inwestycyjne podejmuje bank, towarzystwo ubezpieczeniowe lub zarejestrowany doradca inwestycyjny lub jeżeli suma aktywów programu przekracza 5 mln USD; organizacja charytatywna, korporacja lub spółka partnerska z majątkiem przekraczającym 5 milionów U

In [42]:
with open('saved/related.txt', 'w') as f:
    for line in related_sentences:
        f.write(line + '\n')

In [43]:
def prepare_unrelated_sentences(query_ids, sep_token='|'):
    results = []
    for q_id in query_ids:
        qa_related = list(qa_df[qa_df['query-id'] == q_id]['corpus-id'])
        random_corp_id = random.choice(ds_qa['train']['corpus-id'])
        while random_corp_id in qa_related: # to assure that related text with be taken
            random_corp_id = random.choice(ds_qa['train']['corpus-id'])

        q_text = queries_df[queries_df['_id'] == str(q_id)]['text'].iloc[0]
        unrel_text = corpus_df[corpus_df['_id'] == str(random_corp_id)]['text'].iloc[0]
        results.append(q_text + sep_token + unrel_text)

    return results

In [44]:
N2 = 5000 # ratio of 'positive' to 'negative' will be 1:2
unrelated_sentences = prepare_unrelated_sentences(random.sample(list(query_ids_set), N2), polbert_separator)
unrelated_sentences[:3]

['Spółka chce sprzedać wszystkie swoje aktywa, warte więcej niż cena akcji?[SEP]Moje doświadczenie (z krajów europejskich, ale nie konkretnie Portugalii) jest takie, że lepiej zmienić się w kraju europejskim, ponieważ wiele banków oczywiście da ci dolary amerykańskie, podczas gdy w USA (jest to wyspiarskie miejsce) może być dość trudno znaleźć miejsce do wymiany pieniędzy poza międzynarodowym lotniskiem. Właściwie zostało mi kilkaset euro z ostatniej podróży sprzed kilku lat. Spodziewałem się, że odbędą kolejną podróż, która się nie udała i nie znalazłem miejsca, aby je wymienić. PS: Gwoli informacji, w czasie, gdy pracowałem w Europie i stwierdziłem, że zdecydowanie najłatwiejszym sposobem na przeniesienie części mojej pensji do domu jest otrzymanie 100-dolarowych banknotów z mojego europejskiego banku. Innym sposobem było wypłata pieniędzy z bankomatu, ponieważ banki amerykańskie i europejskie były w tej samej sieci. Niestety IRS położył temu kres, chociaż nie wiem, czy to były wszys

In [45]:
with open('saved/unrelated.txt', 'w') as f:
    for line in unrelated_sentences:
        f.write(line + '\n')

In [59]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dkleczek/bert-base-polish-cased-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(60000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [60]:
related_sentences_df = pd.DataFrame({
                        "text": related_sentences,
                        "label": [1 for _ in range(len(related_sentences))],
                    })

related_sentences_df.head()

,text,label
0,Kiedy powinienem zacząć oszczędzać/inwestować ...,1
1,W jaki sposób obywatel spoza USA uzyskuje stat...,1
2,Kanadyjczyk przyjmujący pieniądze drogą elektr...,1
3,"Jaka jest najlepsza inwestycja, która jest w p...",1
4,Czy są jakieś statystyki potwierdzające potrze...,1


In [61]:
unrelated_sentences_df = pd.DataFrame({
                        "text": unrelated_sentences,
                        "label": [0 for _ in range(len(unrelated_sentences))],
                    })

unrelated_sentences_df.head()

,text,label
0,"Spółka chce sprzedać wszystkie swoje aktywa, w...",0
1,Raport kredytowy – Nie można ustalić tożsamośc...,0
2,Przedpłata pożyczki: Czy odsetki nie powinny b...,0
3,Kiedy zostaną wydane formularze podatku federa...,0
4,Zapytania o akcje[SEP]„Dodam jeszcze jedną poz...,0


In [62]:
dataset = pd.concat([related_sentences_df[:], unrelated_sentences_df[:]], ignore_index=True)
dataset

,text,label
0,Kiedy powinienem zacząć oszczędzać/inwestować ...,1
1,W jaki sposób obywatel spoza USA uzyskuje stat...,1
2,Kanadyjczyk przyjmujący pieniądze drogą elektr...,1
3,"Jaka jest najlepsza inwestycja, która jest w p...",1
4,Czy są jakieś statystyki potwierdzające potrze...,1
...,...,...
7495,Pożyczka osobista dla przyjaciela procedura[SE...,0
7496,Czy podatek obrotowy od zakupów online jest op...,0
7497,Dochodzenie dochodów/odliczeń z nielegalnego m...,0
7498,Dlaczego karty American Express nie są tak pop...,0


In [63]:
from sklearn.model_selection import train_test_split

# train:val:test proportion will be 75:15:10
train_data, temp_data = train_test_split(dataset, test_size=0.25, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.4, random_state=42)

train_data.to_csv("saved/train.csv", index=False)
val_data.to_csv("saved/val.csv", index=False)
test_data.to_csv("saved/test.csv", index=False)

In [64]:
from datasets import load_dataset

data_files = {
    "train": "saved/train.csv",
    "validation": "saved/val.csv",
    "test": "saved/test.csv"
}
raw_datasets = load_dataset("csv", data_files=data_files)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256) # max_lenght set to 256 which seems to be reasonable

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)


Generating train split: 5625 examples [00:00, 70777.89 examples/s]
Generating validation split: 1125 examples [00:00, 70440.41 examples/s]
Generating test split: 750 examples [00:00, 66593.17 examples/s]
Map: 100%|███████████████████████████| 750/750 [00:00<00:00, 8523.99 examples/s]


In [65]:
tokenized_datasets['train']

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5625
})

In [66]:
example = tokenized_datasets["train"][0]
print(f"{example['text'] = }")
print(f"{example['input_ids'] = }")
print(f"{example['attention_mask'] = }")

example['text'] = 'Czy osoby fizyczne mogą handlować akcjami w trybie dziennym przy użyciu handlu wysokiej częstotliwości (HFT)?[SEP]Coś, co chciałbym mocno zakorzenić w twoim umyśle - Jeśli jesteś w stanie zaoszczędzić wystarczająco dużo pieniędzy, aby kupić rzeczy, których chcesz od razu, kredyt na niewiele ci się przyda. Wiele osób po zgromadzeniu bardzo dobrych wyników kredytowych dzięki dobrym nawykom finansowym stwierdza, że \u200b\u200brzadko korzystają z kredytu i niewiele zyskują z posiadania „świetnej” oceny kredytowej w porównaniu z „przeciętną” oceną kredytową. Oczywiście wiele z tego zależałoby od Twojej sytuacji finansowej, ale warto o tym pamiętać. Jak stwierdzili inni i szeroko udokumentowano online, nie musisz dokonywać spłat pożyczki ani nosić salda karty, aby budować swoją historię kredytową. Sprawdź swój kredyt na popularnej stronie, takiej jak Credit Karma (brak przynależności). Tam zobaczysz szczegółowy podział różnych obszarów Twojego profilu kredytowego, które m

In [67]:
print("|".join(tokenizer.convert_ids_to_tokens(list(example["input_ids"]))))

[CLS]|Czy|osoby|fizyczne|mogą|handlowa|##ć|akcja|##mi|w|trybie|dziennym|przy|użyciu|handlu|wysokiej|częstotliwości|(|H|##FT|)|?|[SEP]|Coś|,|co|chciałbym|mocno|zakorze|##nić|w|twoim|umyśle|-|Jeśli|jesteś|w|stanie|zaoszczędzić|wystarczająco|dużo|pieniędzy|,|aby|kupić|rzeczy|,|których|chcesz|od|razu|,|kredyt|na|niewiele|ci|się|przyda|.|Wiele|osób|po|zgromadzeniu|bardzo|dobrych|wyników|kredytowych|dzięki|dobrym|nawyk|##om|finansowym|stwierdza|,|że|rzadko|korzystają|z|kredytu|i|niewiele|zysku|##ją|z|posiadania|„|świetne|##j|”|oceny|kredytowej|w|porównaniu|z|„|przecię|##tną|”|oceną|kredytową|.|Oczywiście|wiele|z|tego|zależało|##by|od|Twojej|sytuacji|finansowej|,|ale|warto|o|tym|pamiętać|.|Jak|stwierdzili|inni|i|szeroko|udokument|##owano|online|,|nie|musisz|dokonywać|spła|##t|pożyczki|ani|nosić|sal|##da|karty|,|aby|budować|swoją|historię|kredytową|.|Sprawdź|swój|kredyt|na|popularnej|stronie|,|takiej|jak|Cre|##di|##t|Karma|(|brak|przynależności|)|.|Tam|zobaczysz|szczegółowy|podział|różnych|obs

In [68]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# learning_rate, and batch sized choosen based on https://github.com/google-research/bert
# num_train_epochs set up to 5 (previously 3) to see if it can improve anything
# the rest of the parameters remain default

In [69]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
import numpy as np

def compute_metrics(pred):
    predictions = np.argmax(pred.predictions, axis=1)
    labels = pred.label_ids
    acc = accuracy_score(labels, predictions)
    auc = roc_auc_score(labels, predictions)
    precision, recall, f1, support = precision_recall_fscore_support(labels, predictions, average="binary")
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1, 'auc': auc}


In [70]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipykernel_15174/564752669.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [71]:
trainer.train()

/home/mklimek/venvs/pytorch-env/lib/python3.12/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/home/mklimek/venvs/pytorch-env/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Auc
1,No log,0.302986,0.879111,0.806295,0.856041,0.830424,0.873673
2,No log,0.330548,0.884444,0.804706,0.879177,0.840295,0.883203
3,0.293900,0.347652,0.891556,0.852243,0.830334,0.841146,0.877124
4,0.293900,0.499339,0.882667,0.876833,0.768638,0.819178,0.855786
5,0.293900,0.528900,0.890667,0.869444,0.804627,0.835781,0.870384


/home/mklimek/venvs/pytorch-env/lib/python3.12/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/home/mklimek/venvs/pytorch-env/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/mklimek/venvs/pytorch-env/lib/python3.12/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/home/mklimek/venvs/pytorch-env/lib/python3.12/site-packages/to

TrainOutput(global_step=880, training_loss=0.1968797423622825, metrics={'train_runtime': 738.1487, 'train_samples_per_second': 38.102, 'train_steps_per_second': 1.192, 'total_flos': 3699999216000000.0, 'train_loss': 0.1968797423622825, 'epoch': 5.0})

In [72]:
validation_metrics = trainer.evaluate(tokenized_datasets["validation"])
print("Validation Metrics:", validation_metrics)

test_metrics = trainer.evaluate(tokenized_datasets["test"])
print("Test Metrics:", test_metrics)

/home/mklimek/venvs/pytorch-env/lib/python3.12/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/home/mklimek/venvs/pytorch-env/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Validation Metrics: {'eval_loss': 0.3476518392562866, 'eval_accuracy': 0.8915555555555555, 'eval_precision': 0.8522427440633246, 'eval_recall': 0.8303341902313625, 'eval_f1': 0.8411458333333334, 'eval_auc': 0.8771236168548115, 'eval_runtime': 10.3205, 'eval_samples_per_second': 109.006, 'eval_steps_per_second': 3.488, 'epoch': 5.0}


/home/mklimek/venvs/pytorch-env/lib/python3.12/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/home/mklimek/venvs/pytorch-env/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Test Metrics: {'eval_loss': 0.3994769752025604, 'eval_accuracy': 0.872, 'eval_precision': 0.8384615384615385, 'eval_recall': 0.8014705882352942, 'eval_f1': 0.8195488721804511, 'eval_auc': 0.8568022397243417, 'eval_runtime': 6.9299, 'eval_samples_per_second': 108.226, 'eval_steps_per_second': 3.463, 'epoch': 5.0}


In [73]:
trainer.save_model("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [75]:
from transformers import pipeline

classifier = pipeline("text-classification", model="./fine_tuned_model", tokenizer="./fine_tuned_model", device='cuda')

In [103]:
result = classifier([example['text']])
print(result)

[{'label': 'LABEL_0', 'score': 0.9959685802459717}]


In [79]:
# ES code based on lab 2

from elasticsearch import Elasticsearch

es = Elasticsearch(
    hosts=["http://localhost:9200"],
    request_timeout=60
)


In [84]:
analyzers_config = {
    "settings": {
        "analysis": {
            "analyzer": {
                "morfologik_analyzer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "morfologik_stem",
                        "lowercase"
                    ]
                }
            }
        }
    }
}

In [90]:
analyzers_config["mappings"] = {"properties": {
                                    "fiqa_id": {
                                            "type": "keyword"
                                    },
                                    "fiqa_morfologik": {
                                        "type": "text",
                                        "analyzer": "morfologik_analyzer"
                                    }
                                }}


In [91]:
index_name = 'fiqa_index'

if es.indices.exists(index=index_name):
    print(f"index {index_name} already exists")
    es.indices.delete(index=index_name)

es.indices.create(index=index_name, body=analyzers_config)

index fiqa_index already exists


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'fiqa_index'})

In [92]:
settings = es.indices.get_settings(index=index_name)
settings

ObjectApiResponse({'fiqa_index': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'fiqa_index', 'creation_date': '1732483046640', 'analysis': {'analyzer': {'morfologik_analyzer': {'filter': ['lowercase', 'morfologik_stem', 'lowercase'], 'type': 'custom', 'tokenizer': 'standard'}}}, 'number_of_replicas': '1', 'uuid': 'InbN0J6xT-2QUspH7shcCQ', 'version': {'created': '8512000'}}}}})

In [93]:
mappings = es.indices.get_mapping(index=index_name)
mappings

ObjectApiResponse({'fiqa_index': {'mappings': {'properties': {'fiqa_id': {'type': 'keyword'}, 'fiqa_morfologik': {'type': 'text', 'analyzer': 'morfologik_analyzer'}}}}})

In [95]:
documents_to_add = [
    {
        "_index": index_name, 
        "_source": {
            "fiqa_id": sample['_id'],
            "fiqa_morfologik": sample['text'],
        }
    }
    for sample in ds_corpus['corpus']
]

In [97]:
documents_to_add[0]

{'_index': 'fiqa_index',
 '_source': {'fiqa_id': '3',
  'fiqa_morfologik': 'Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. Szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. Być może systemy edukacyjne w Stanach Zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłużonych studentów i narzekać, że nie są do niczego wykwalifikowani.'}}

In [99]:
from elasticsearch import helpers

def batch_data(documents, batch_size=5000):
    for i in range(0, len(documents), batch_size):
        yield documents[i:i + batch_size]

for batch in batch_data(documents_to_add, batch_size=5000):
    success, failed = helpers.bulk(es, batch, raise_on_error=False,  ignore_status=[400, 404], timeout="120s")
    print(f"{success} texts added succesfully.")

5000 texts added succesfully.
5000 texts added succesfully.
5000 texts added succesfully.
5000 texts added succesfully.
5000 texts added succesfully.
5000 texts added succesfully.
5000 texts added succesfully.
5000 texts added succesfully.
5000 texts added succesfully.
5000 texts added succesfully.
5000 texts added succesfully.
2638 texts added succesfully.


In [100]:
simplified_dict = {q_id: [] for q_id in set(ds_qa["test"]['query-id'])}
simplified_query_test_only = {}

for i in range(len(ds_qa["test"])):
    qa = ds_qa["test"][i]
    simplified_dict[qa["query-id"]].append(qa["corpus-id"])

for i in range(len(ds_queries['queries'])):
    if int(q_id := ds_queries['queries'][i]['_id']) in simplified_dict:
        simplified_query_test_only[int(q_id)] = ds_queries['queries'][i]['text']

In [101]:
import numpy as np

def ndcg5(hits_ids, relevant_ids):
    ideal_dcg = 0
    dcg = 0
    
    for i in range(5):
        if hits_ids[i] in relevant_ids:   
            dcg += 1/np.log2(i+2)

    for i in range(len(relevant_ids)):
        ideal_dcg += 1/np.log2(i+2)

    return dcg/ideal_dcg


In [154]:
def find_most_relevant_with_polbert(query, texts, separator, n=5):
    texts_with_scores = []

    model_texts = [query + separator + text for _, text in texts]
    truncated_texts = []
    
    for text in model_texts:
        tokens = tokenizer(text, truncation=True, max_length=256, return_tensors="pt")
        truncated_texts.append(tokenizer.decode(tokens["input_ids"][0]))

    results = classifier(truncated_texts)
    
    for i in range(len(results)):
        if results[i]['label'] == "LABEL_0":
            score = 1 - results[i]['score']
        else: 
            score = results[i]['score']

        texts_with_scores.append((texts[i][0], score))

    sorted_scores = sorted(texts_with_scores, key=lambda x: x[1], reverse=True)
    return sorted_scores[:n]

In [155]:
def calc_ndcg(source):
    ndcgs = []
    N = 100
    
    for q_id in set(ds_qa["test"]['query-id']):
        query = simplified_query_test_only[q_id]
        relevant_ids = simplified_dict[q_id]
        
        response = es.search(
            index=index_name,
            body={
                "query": {
                    "match": {
                        source: query
                    }
                },
                "_source": ["fiqa_id", "fiqa_morfologik"],
                "from": 0,
                "size": N  
            }
        )

        hit_texts = [(int(hit['_source']['fiqa_id']), hit['_source']['fiqa_morfologik']) 
                     for hit in response['hits']['hits']]
        
        polbert_best = find_most_relevant_with_polbert(query, hit_texts, polbert_separator)
        hit_ids = [scoring[0] for scoring in polbert_best]
        ndcgs.append(ndcg5(hit_ids, relevant_ids))

    return np.mean(ndcgs)

In [153]:
mean_ndcg = calc_ndcg("fiqa_morfologik")
print(f"ndcg with polbert on 100 best ES matchings - {mean_ndcg}")

ndcg with polbert on 100 best ES matchings - 0.12113128082951996


In [156]:
# results are worse than anything that was achieved with pure ES :(
# best results achieved in ES exercise was ndcg = 0.182

# modification of calc_ndcg to add some prints with polbert results and using olny first query 
# it turns out that almost all from top 100 sentences return by ES 
# are labeled as 'LABEL_1' with very high scores (over 0.99)

mean_ndcg = calc_ndcg("fiqa_morfologik")

{'label': 'LABEL_1', 'score': 0.9972023963928223}
{'label': 'LABEL_1', 'score': 0.9972135424613953}
{'label': 'LABEL_1', 'score': 0.9970248341560364}
{'label': 'LABEL_1', 'score': 0.9972267746925354}
{'label': 'LABEL_1', 'score': 0.9964908957481384}
{'label': 'LABEL_1', 'score': 0.9959766268730164}
{'label': 'LABEL_1', 'score': 0.9964867830276489}
{'label': 'LABEL_1', 'score': 0.9963847398757935}
{'label': 'LABEL_1', 'score': 0.9965294003486633}
{'label': 'LABEL_1', 'score': 0.9971676468849182}
{'label': 'LABEL_1', 'score': 0.9969710111618042}
{'label': 'LABEL_1', 'score': 0.9952327609062195}
{'label': 'LABEL_1', 'score': 0.9960582256317139}
{'label': 'LABEL_1', 'score': 0.996483325958252}
{'label': 'LABEL_1', 'score': 0.9969987869262695}
{'label': 'LABEL_1', 'score': 0.9970510005950928}
{'label': 'LABEL_1', 'score': 0.9955151677131653}
{'label': 'LABEL_1', 'score': 0.9968955516815186}
{'label': 'LABEL_1', 'score': 0.9967095851898193}
{'label': 'LABEL_1', 'score': 0.9972556233406067}
{

## Wnioski
Niestety, po zastosowaniu w zadaniu dodatkowego modelu językowego do klasyfikacji, wyniki uzyskiwane w metryce ndcg@5, w porównaniu z wynikami uzyskiwanymi w labolatorium dotyczącym ES, uległy pogorszeniu. (Sam ES uzyskiwał ndcg@5 na poziomie 0.182, po zastosowaniu Polberta wynik w tej metryce to 0.121). 
Możliwe powody:
- niskiej jakości zbiór danych i zbyt mała wybrana próbka
- nieodpowiednie zbalansowanie danych w zbiorze treningowym - tylko dwukrotnie więcej próbek negatywnych w stosunku do pozytywnych, co nie oddawało proporcji w rzeczywistym zbiorze,  gdzie tylko kilka tekstów jest adekwatnych do pojedynczego zapytania

## Odpowiedzi na pytania

Prostsze metody, takie jak bag-of-words raczej nie byłyby użyteczne w tego typu zastosowaniach, ponieważ nie miałyby one szansy zobaczyć podziału na pytanie i odpowiedź, które są oddzielone specjalnym separatorem - w czasie zliczania słów w BOW uwzględniane byłyby zarówno elementy z pytania, jak i te z potencjalnej odpowiedzi.


W zadaniu użyty został model [Polbert](https://huggingface.co/dkleczek/bert-base-polish-cased-v1), który bazuje na modelu bert-base-cased. Hiperparametry do treningu modelu zostały wybrane na podstawie [pliku README z repozytorium google](https://github.com/google-research/bert?tab=readme-ov-file#sentence-and-sentence-pair-classification-tasks). Z tego źródła zaczerpnięte zostały przede wszystkim watości learning_rate oraz num_train_epochs. Ilość epok została zwiększona w czasie przeprowadzania eksperymentu z 3, do 5, ale w użytych metrykach na zbiorze walidacyjnym nie przyniosło poprawy wyników. Z [tego samego źródła](https://github.com/google-research/bert?tab=readme-ov-file#out-of-memory-issues) pozyskane zostały też informacje, jak powinna być dobrana maksymalna długość sekwencji oraz batch_size dla kart graficznych, na których odbywał się trening modelu.


Zalety sieci neuronowych w NLP:
- pozwalają wykryć znacznie bardziej skomplikowane powiązania pomiędzy słowami w tekstach, niż jakiekolwiek inne modele
- dzięki bibliotekom takim jak transformers mogą być łatwo dostosowywane do różnych zadań (dodawanie nowych tokenów na końcu zdań, klasyfikacja, uzupełnianie luk w tekście)

Wady sieci neuronowych w NLP:
- aby były efektywne, wymagają olbrzymich ilości wysokiej jakości danych, które ciężko jest pozyskać i zweryfikować
- trening takich modeli oraz ich używanie wymaga dużych mocy obliczeniowych, które są drogie w zakupie i zużywają ogromne ilości energii
- kontrola nad procesem uczenia i możliwość pełnego przetestowania modelu jest dość ograniczona, dlatego model może w niektórych sytuacjach zwracać niespodziewane wyniki (np. zapamiętywać wiadomości, które są prawdziwe tylko w określonych warunkach)
